<a href="https://colab.research.google.com/github/ozgurmelih/ainanda-text-sentiment-analysis/blob/main/ainanda_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AInanda inference Demo

A quick intro to using the pre-trained AInanda model to detect sentiment of text.

##Install required packages

In [ ]:
# Scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 


# Word2vec
import gensim
from gensim.models import Word2Vec

import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools
import pandas as pd

import warnings 
warnings.filterwarnings("ignore")

##Download and extract Pretrained model
Download the pretrained  model from google drive

In [ ]:
fileId = '146mZJzhphx77nF98UoX-jC-PM2b1ikL6'  #ID code of pretrained model file

from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#OAouth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#code for download file from google drive
fileName = 'pretrained_model.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)

#Extract zip file
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

Extracted zip file pretrained_model.zip


##Create Model and Load Trained Weights

In [ ]:
#load the pretrained weights
w2v_model = Word2Vec.load('/content/pretrained_model/ainanda_model_v1.w2v') 
model = load_model('/content/pretrained_model/ainanda_model_v1.h5') 
with open('/content/pretrained_model/tokenizer.pkl', 'rb') as handle: tokenizer = pickle.load(handle) 
with open('/content/pretrained_model/encoder.pkl', 'rb') as handle: encoder = pickle.load(handle)

In [ ]:
# This function enables us to predict the labels of the text input on the basis of the trained model.

def predict_text(text):
  x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=300) # Tokenize text
  score = model.predict([x_test])[0] #
  
  if score <= 0.4:
      label = 'Negative'
      emoji = '\U0001F641'  #Unicode symbol for negative
  elif score >= 0.6:
        label = 'Positive'
        emoji = '\U0001F60A' #Unicode symbol for positive
  else:        
          label = 'Neutral'
          emoji = '\U0001F610' #Unicode symbol for neutral
 
  return {"Label": label, "Score": float(score), "Emoji":emoji}



Run text sentiment analysis

In [ ]:
#Run detection
predict_text("I don't think there is anything I dislike about the product")

{'Emoji': '😊', 'Label': 'Positive', 'Score': 0.9715813398361206}

##GUI demo 



In [ ]:
#to start using the library we need to install the ipywidgets extension
!pip install -q ipywidgets

In [ ]:
from ipywidgets import interact, widgets
from IPython.display import display

interact(predict_text, text='')

interactive(children=(Text(value='', description='text'), Output()), _dom_classes=('widget-interact',))

<function __main__.predict_text>